# Preprocessing

In [1]:
import requests
import datetime

In [2]:
def get_historical_data_with_end_data(start_date, finish_date, ticker,
                                      datablock='history', engine='stock', market='shares', board='TQBR',
                                      columns='TRADEDATE, OPEN, CLOSE'):
    """To get historical stocks data from specific till specific date.
    Accept only one ticker. Return list of data: date, open, close.
    :argument start_date: should be in string format like '2023-01-01'
    :argument finish_date: should be in string format like '2023-01-03'
    :argument ticker: accept only one ticker"""

    result = []

    finish_date = datetime.datetime.strptime(finish_date, '%Y-%m-%d').date()
    days = [finish_date - datetime.timedelta(days=i) for i in range(4)]
    start_date = datetime.datetime.strptime(start_date, '%Y-%m-%d').date()

    def get_data(start_date=start_date, end_date=None):
        URL = f'http://iss.moex.com/iss/history/engines/{engine}/markets/{market}/boards/{board}/securities/{ticker}.json'
        r = requests.get(URL, params={'iss.meta': 'off', 'iss.only': datablock,
                                      'iss.json': 'extended',
                                      f'{datablock}.columns': columns,
                                      'from': start_date,
                                      'till': end_date})

        data = r.json()[1].get(datablock)
        for i in data:
            for key in i:
                result.append(i[key])

        end_date = data[len(data) - 1].get('TRADEDATE')
        end_date = datetime.datetime.strptime(end_date, '%Y-%m-%d').date()
        return end_date

    while True:

        end_date = start_date + datetime.timedelta(days=100)

        if finish_date <= end_date:
            end_date = finish_date

        if end_date in days:  # ?
            if end_date in result:
                return result
            else:
                get_data(start_date=start_date, end_date=end_date)
                return result
        else:
            end_date = get_data(start_date=start_date, end_date=end_date)
            start_date = end_date

In [3]:
result = get_historical_data_with_end_data(start_date='2022-01-03', finish_date='2023-06-05', ticker='GAZP')

In [ ]:
data = []
temporary = []

for i, v in enumerate(raw_data, 1):
    temporary.append(v)
    
    if i % 3 == 0:
        data.append(temporary)
        temporary = []

In [ ]:
df = pd.DataFrame(columns=['date', 'open', 'close'], data=data)
df.drop(df[df['close'] == 'None'].index, inplace=True)
df = df.reset_index(drop=True)

df = df.rename(columns={'close': 'y', 'date': 'ds'})
df = df.drop(columns=['open'])

df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].astype('float')

df['y_18'] = df.iloc[:,1].rolling(window=18).mean().round(2)
df['y_50'] = df.iloc[:,1].rolling(window=50).mean().round(2)

display(df.tail())
df.info()

<b>Import Libs</b>

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet 
import matplotlib.pyplot as plt
 
%matplotlib inline

plt.rcParams['figure.figsize']=(20,10)
plt.style.use('ggplot')

<b>Read *csv file</b>

In [ ]:
f= open('API data/MTSS.csv', 'r')
raw_data = f.read().split(',')

data = []
temporary = []

for i, v in enumerate(raw_data, 1):
    temporary.append(v)
    
    if i % 3 == 0:
        data.append(temporary)
        temporary = []

<b>Make DF and preprocessing data</b>

In [ ]:
df = pd.DataFrame(columns=['date', 'open', 'close'], data=data)
df.drop(df[df['close'] == 'None'].index, inplace=True)
df = df.reset_index(drop=True)

df = df.rename(columns={'close': 'y', 'date': 'ds'})
df = df.drop(columns=['open'])

df['ds'] = pd.to_datetime(df['ds'])
df['y'] = df['y'].astype('float')

df['y_18'] = df.iloc[:,1].rolling(window=18).mean().round(2)
df['y_50'] = df.iloc[:,1].rolling(window=50).mean().round(2)

display(df.tail())
df.info()

# Let's Start Prophet

<b>Look at the data</b>

In [ ]:
df.set_index('ds').y.plot()
df.set_index('ds').y_18.plot()
df.set_index('ds').y_50.plot()

**Running Prophet**

In [ ]:
model = Prophet()
model.fit(df)

# Added (AM: changed to 30 days [23.06.2023])
future = model.make_future_dataframe(periods=30, freq = 'd')
#future.tail()

#model.fit(df)
forecast = model.predict(future)
fig = model.plot_components(forecast)

In [ ]:
forecast = model.predict(future)

In [ ]:
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail()

<b>Plotting</b>

In [ ]:
model.plot(forecast);

<b>Visualizing Prophet models</b>

In [ ]:
df.set_index('ds', inplace=True)
forecast.set_index('ds', inplace=True)

In [ ]:
viz_df = df.join(forecast[['yhat', 'yhat_lower','yhat_upper']], how = 'outer')

In [ ]:
viz_df[['y', 'yhat','y_18','y_50']].plot()

<b> Saving project data in *xlsx</b>

In [ ]:
viz_df.to_excel("forecast_results/MTSS_forecast.xlsx")  